In [1]:
# Instalar Bibliotecas 
!pip install python-bcb

In [6]:
# Importar Bibliotecas 
from bcb import sgs
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tools.eval_measures import rmse

Coleta de dados

In [8]:
dados_brutos = sgs.get(
    codes = {"caged": 28772},
    end = "2024-06-01"
)
dados_brutos

,caged
Date,
1992-01-01,9900801
1992-02-01,9897618
1992-03-01,9884230
1992-04-01,9878410
1992-05-01,9892180
...,...
2023-11-01,22310194
2023-12-01,22116274
2024-01-01,22191454


Modelagem 

In [17]:
teste = dados_brutos.tail(12)
treino = sm.tsa.add_trend(dados_brutos.query("Date not in @teste.index"), trend = "t")

teste = teste.assign(trend = range(int(treino.trend.max()+1), int(treino.trend.max()+1+12)))

print(treino.tail())
print(teste)

               caged  trend
Date                       
2022-11-01  21452481  371.0
2022-12-01  21245361  372.0
2023-01-01  21288006  373.0
2023-02-01  21459553  374.0
2023-03-01  21582700  375.0
               caged  trend
Date                       
2023-04-01  21688198    376
2023-05-01  21772731    377
2023-06-01  21847751    378
2023-07-01  21903784    379
2023-08-01  22018437    380
2023-09-01  22112663    381
2023-10-01  22220362    382
2023-11-01  22310194    383
2023-12-01  22116274    384
2024-01-01  22191454    385
2024-02-01  22386838    386
2024-03-01  22535560    387


In [18]:
# Estima modelos de exemplo
mqo = smf.ols(formula = "caged ~ trend", data = treino).fit()
ets = ExponentialSmoothing(treino.caged).fit()

c:\Users\pedro\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\pedro\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


Produzindo Previsões 

In [20]:
prev_mqo = mqo.predict(teste)
print(prev_mqo)
prev_ets = ets.forecast(12)
print(prev_ets)

Date
2023-04-01    2.092876e+07
2023-05-01    2.096372e+07
2023-06-01    2.099868e+07
2023-07-01    2.103364e+07
2023-08-01    2.106860e+07
2023-09-01    2.110355e+07
2023-10-01    2.113851e+07
2023-11-01    2.117347e+07
2023-12-01    2.120843e+07
2024-01-01    2.124339e+07
2024-02-01    2.127835e+07
2024-03-01    2.131331e+07
dtype: float64
2023-04-01    2.158208e+07
2023-05-01    2.158208e+07
2023-06-01    2.158208e+07
2023-07-01    2.158208e+07
2023-08-01    2.158208e+07
2023-09-01    2.158208e+07
2023-10-01    2.158208e+07
2023-11-01    2.158208e+07
2023-12-01    2.158208e+07
2024-01-01    2.158208e+07
2024-02-01    2.158208e+07
2024-03-01    2.158208e+07
Freq: MS, dtype: float64


In [22]:
print(treino.caged.tail())
print(mqo.fittedvalues.tail())

Date
2022-11-01    21452481
2022-12-01    21245361
2023-01-01    21288006
2023-02-01    21459553
2023-03-01    21582700
Name: caged, dtype: int64
Date
2022-11-01    2.075396e+07
2022-12-01    2.078892e+07
2023-01-01    2.082388e+07
2023-02-01    2.085884e+07
2023-03-01    2.089380e+07
dtype: float64


In [23]:
# Calcula métricas de acurácia
pd.DataFrame(
    {
        "Amostra": ["Treino", "Treino", "Teste", "Teste"],
        "Modelo": ["MQO", "ETS", "MQO", "ETS"],
        "RMSE": [
            rmse(treino.caged, mqo.fittedvalues), rmse(treino.caged, ets.fittedvalues),
            rmse(teste.caged, prev_mqo), rmse(teste.caged, prev_ets)
            ]
    }
)

,Amostra,Modelo,RMSE
0,Treino,MQO,1.023622e+06
1,Treino,ETS,7.563570e+04
2,Teste,MQO,9.805606e+05
3,Teste,ETS,5.660258e+05
